In [ ]:
import os

# Remove GPU use
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys

sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np

from choice_learn.models.simple_mnl import SimpleMNL
from choice_learn.data import ChoiceDataset
from choice_learn.datasets.base import load_heating

Let's recreate this [tutorial](https://cran.r-project.org/web/packages/mlogit/vignettes/e1mlogit.html) by Yves Croissant for the mlogit R package.

It uses the Heating dataset, where we try to predict which heating harware a houseold will chose. The dataset is integrated in the package, you can find information [here].

In [ ]:
heating_df = load_heating(as_frame=True)

contexts_features = ["income", "agehed", "rooms"]
choice = ["depvar"]
contexts_items_features = ["ic.", "oc."]
items = ["hp", "gc", "gr", "ec", "er"]

choices = np.array([items.index(val) for val in heating_df[choice].to_numpy().ravel()])
contexts = heating_df[contexts_features].to_numpy()
contexts_items = np.stack([heating_df[[feat + item for feat in contexts_items_features]].to_numpy() for item in items], axis=1)

First part estimates a simple MNL without intercept from the 'ic' and 'oc' features. By default, SimpleMNL does not integrate any intercept, but you can precise 'None'.

In [ ]:
dataset = ChoiceDataset(contexts_items_features=contexts_items, choices=choices)
model = SimpleMNL(optimizer="lbfgs", intercept=None)
history = model.fit(dataset, epochs=100, get_report=True)

In [ ]:
print("Estimation Negative LogLikelihood:",
      model.evaluate(dataset) * len(dataset))

In [ ]:
model.report

We reach very similar results. The second part is about modelling useing the ic + oc/0.12 ratio. Here is how it can be done:

In [ ]:
ratio_contexts_items = []
for case in range(contexts_items.shape[0]):
    feat = []
    for item in range(contexts_items.shape[1]):
        feat.append([contexts_items[case, item, 0] + contexts_items[case, item, 1] / 0.12])
    ratio_contexts_items.append(feat)
ratio_contexts_items = np.array(ratio_contexts_items)
ratio_contexts_items.shape

In [ ]:
ratio_dataset = ChoiceDataset(contexts_items_features=ratio_contexts_items, choices=choices)
model = SimpleMNL(optimizer="lbfgs")
history = model.fit(ratio_dataset, epochs=100, get_report=False)

In [ ]:
print("Weights:", model.weights)
print("Estimation Negative LogLikelihood:", model.evaluate(ratio_dataset) * len(ratio_dataset))

Finally, to add itemwise intercept for the last part, here is how it can be done:

In [ ]:
model = SimpleMNL(optimizer="lbfgs", intercept="item")
history = model.fit(dataset, epochs=100, get_report=True)

In [ ]:
model.report

In [ ]:
from choice_learn.datasets import load_swissmetro
from choice_learn.models.base_model import BaseMixtureModel

In [ ]:
sm_df = load_swissmetro(as_frame=True)
sm_df = sm_df.loc[sm_df.CHOICE != 0]
sm_df.head()

In [ ]:
contexts_items = np.stack([sm_df[[f"{item}_{feature}" for feature in ["CO", "TT"]]].to_numpy() for item in ["CAR", "TRAIN", "SM"]],
axis=1)
contexts_items[:, :, 0] = contexts_items[:, :, 0] / 100
contexts_items[:, :, 1] = contexts_items[:, :, 1] / 100
dataset = ChoiceDataset(
    contexts_items_features=(contexts_items, ),
    choices=sm_df.CHOICE.to_numpy()-1,
    contexts_items_features_names=(["CO", "TT"], ),
)

In [ ]:
np.unique(dataset.choices), len(dataset)

In [ ]:
import tensorflow as tf
import importlib
import choice_learn.models.base_model
importlib.reload(choice_learn.models.base_model)

from choice_learn.models.base_model import BaseMixtureModel
from choice_learn.models.conditional_mnl import ConditionalMNL

In [ ]:
spec = {"CO": "constant", "TT": "item", "intercept": "item-full"}
spec2 = {"CO": "constant", "intercept": "item-full"}

In [ ]:
model = BaseMixtureModel(
    n_latent_classes=4,
    model_class=ConditionalMNL,
    model_parameters={"optimizer": "lbfgs", "epochs": 100,"parameters": spec},
    fit_method="EM",
    epochs=10,
)
model.instantiate()

In [ ]:
type(dataset.contexts_items_features)

In [ ]:
model._em_fit(dataset)

In [ ]:
tf.multiply(dataset.contexts_items_features[0][:, :, 0], model.models[0].weights[0])

In [ ]:
model.models[1].fit(dataset)

In [ ]:
dataset.contexts_items_features[0].shape

In [ ]:
model.models[0].weights

In [ ]:
model.latent_logits

In [ ]:
model.models[1].weights

In [ ]:
model.models[0].weights

In [ ]:
model.models[0].predict_probas(dataset)

In [ ]:
model.models[1].predict_probas(dataset)

In [ ]:
w = model._expectation(dataset)

In [ ]:
np.sum(w, axis=0)

In [ ]:
model.weights[:10]

In [ ]:
hist = model._em_fit(dataset)

In [ ]:
model.latent_logits

In [ ]:
model.weights

In [ ]:
plt.plot(np.array(hist)[:, 0])
plt.plot(np.array(hist)[:, 1])

In [ ]:
model.models[0].weights, model.models[1].weights

In [ ]:
model.weights, model.latent_logits, l

In [ ]:
a

In [ ]:
model.latent_logits = l
w = model._expectation(dataset)

In [ ]:
w.shape

In [ ]:
w

In [ ]:
model = SimpleMNL(optimizer="lbfgs", intercept=None)
model.fit(dataset, sample_weight=np.concatenate([[1.], [.0]*(len(dataset)-1)]))

In [ ]:
model.predict_probas(dataset)[0]

In [ ]:
model = SimpleMNL(optimizer="lbfgs", intercept=None)
model.fit(dataset, sample_weight=np.concatenate([[0.], [.1]*(len(dataset)-1)]))
model.weights

In [ ]:
model.predict_probas(dataset)[:10], np.min(model.predict_probas(dataset)), np.max(model.predict_probas(dataset))

In [ ]:
model = SimpleMNL(optimizer="lbfgs", intercept=None)
model.fit(dataset)
model.predict_probas(dataset)[:10]

In [ ]:
model.instantiate()
model._em_fit(dataset)

In [ ]:
model.models[0].weights

In [ ]:
model.models[1].weights

In [ ]:
model.latent_logits

In [ ]:

predicted_probas = [m.predict_probas(dataset) for m in model.models]


In [ ]:
predicted_probas

In [ ]:
for latent, proba in zip(model.latent_logits, predicted_probas):
    print(latent, proba[:4])
    print(latent
    * tf.gather_nd(
        params=proba,
        indices=tf.stack([tf.range(0, len(dataset), 1), dataset.choices], axis=1),
    )[:4])

In [ ]:
predicted_probas = [
    latent
    * tf.gather_nd(
        params=proba,
        indices=tf.stack([tf.range(0, len(dataset), 1), dataset.choices], axis=1),
    )
    for latent, proba in zip(model.latent_logits, predicted_probas)
]


In [ ]:
predicted_probas

In [ ]:

weights = predicted_probas / tf.reduce_sum(predicted_probas, axis=0, keepdims=True)
weights

In [ ]:
for q in range(self.n_latent_classes):
    self.models[q].fit(dataset, sample_weight=weights[q])

self.latent_logits = tf.reduce_mean(weights, axis=1)

In [ ]:
model = SimpleMNL(optimizer="lbfgs", intercept="item")
model.fit(dataset, sample_weight=np.random.rand(len(dataset)))

In [ ]:
model.weights

In [ ]:
model = SimpleMNL(optimizer="lbfgs", intercept="item")
model.fit(dataset)
model.weights

In [ ]:
predicted_probas = np.array([[[.1, .1, .8], [.5, .4, .1]], [[.2, .3, .5], [.3, .3, .4]]])
latent_probas = [.3, .7]

In [ ]:

predicted_probas = np.concatenate([latent_probas[q]*predicted_probas[q][np.arange(2), [1, 2]].reshape((-1,1)) for q in range(2)], axis = 1)

In [ ]:
print(predicted_probas)
weights = predicted_probas/np.sum(predicted_probas,axis = 1,keepdims=True)
print(weights)